# ETL 설계 2편: Bronze 레이어 - 원본을 있는 그대로 쌓는다

Full Load와 Incremental Load의 차이, 하이 워터마크 패턴, 메타데이터 컬럼 설계를 직접 실습한다.

📖 [블로그 원문](https://biz-agentic-ai.github.io/guides/etl-design/002-bronze-layer/)

## 환경 세팅

In [ ]:
# 도구 설치 (protobuf 경고는 무시해도 됩니다 - Colab의 tensorflow와 버전 충돌이지만 실습에 영향 없음)
!pip install -q duckdb 2>&1 | grep -v "protobuf"

In [ ]:
import duckdb

conn = duckdb.connect('warehouse.duckdb')
conn.execute("CREATE SCHEMA IF NOT EXISTS bronze;")
print(f"DuckDB {duckdb.__version__} 준비 완료")

## Full Load 시뮬레이션

Full Load는 소스 테이블 전체를 매번 가져와서 덮어쓴다.

`CREATE OR REPLACE TABLE`이 핵심이다. 매번 테이블을 새로 만든다.

In [ ]:
# Full Load: 통째로 교체
conn.execute("""
CREATE OR REPLACE TABLE bronze.orders AS
SELECT *
FROM read_csv_auto(
  'https://raw.githubusercontent.com/dbt-labs/jaffle_shop/main/seeds/raw_orders.csv'
);
""")

cnt = conn.execute("SELECT count(*) FROM bronze.orders").fetchone()[0]
print(f"Full Load 완료: {cnt}건")

In [ ]:
# Full Load 결과 확인
conn.execute("SELECT * FROM bronze.orders LIMIT 5").fetchdf()

## Incremental Load 시뮬레이션

마지막으로 가져온 지점을 기억해야 한다. 이 패턴을 **하이 워터마크(High Watermark)** 라고 부른다.

In [ ]:
# 워터마크 테이블: 마지막 적재 지점을 기록
conn.execute("""
CREATE TABLE IF NOT EXISTS bronze.watermarks (
    table_name VARCHAR PRIMARY KEY,
    last_loaded_id INTEGER,
    last_loaded_at TIMESTAMP DEFAULT current_timestamp
);
""")

# 현재 워터마크 확인
watermark = conn.execute("""
SELECT COALESCE(last_loaded_id, 0)
FROM bronze.watermarks
WHERE table_name = 'orders'
""").fetchone()

last_id = watermark[0] if watermark else 0
print(f"마지막 적재 ID: {last_id}")

In [ ]:
# 시뮬레이션을 위해 일부 데이터만 먼저 적재 (id <= 50)
conn.execute("""
CREATE OR REPLACE TABLE bronze.orders_incremental AS
SELECT *
FROM read_csv_auto(
  'https://raw.githubusercontent.com/dbt-labs/jaffle_shop/main/seeds/raw_orders.csv'
)
WHERE id <= 50;
""")

# 워터마크 기록
conn.execute("""
INSERT OR REPLACE INTO bronze.watermarks (table_name, last_loaded_id, last_loaded_at)
SELECT 'orders_incremental', MAX(id), current_timestamp
FROM bronze.orders_incremental;
""")

cnt = conn.execute("SELECT count(*) FROM bronze.orders_incremental").fetchone()[0]
wm = conn.execute("SELECT last_loaded_id FROM bronze.watermarks WHERE table_name = 'orders_incremental'").fetchone()[0]
print(f"1차 적재: {cnt}건, 워터마크: {wm}")

In [ ]:
# 증분 적재: 워터마크 이후 데이터만 가져온다
last_id = conn.execute("""
SELECT last_loaded_id FROM bronze.watermarks
WHERE table_name = 'orders_incremental'
""").fetchone()[0]

conn.execute(f"""
INSERT INTO bronze.orders_incremental
SELECT *
FROM read_csv_auto(
  'https://raw.githubusercontent.com/dbt-labs/jaffle_shop/main/seeds/raw_orders.csv'
)
WHERE id > {last_id};
""")

# 워터마크 갱신
conn.execute("""
INSERT OR REPLACE INTO bronze.watermarks (table_name, last_loaded_id, last_loaded_at)
SELECT 'orders_incremental', MAX(id), current_timestamp
FROM bronze.orders_incremental;
""")

cnt = conn.execute("SELECT count(*) FROM bronze.orders_incremental").fetchone()[0]
wm = conn.execute("SELECT last_loaded_id FROM bronze.watermarks WHERE table_name = 'orders_incremental'").fetchone()[0]
print(f"증분 적재 완료: 총 {cnt}건, 워터마크: {wm}")

## 메타데이터 컬럼 추가

원본 컬럼은 그대로 두고, 적재 시점·소스 시스템·적재 방식을 메타데이터로 붙인다.

언더스코어(`_`)로 시작해서 원본 컬럼과 구분한다.

In [ ]:
conn.execute("""
CREATE OR REPLACE TABLE bronze.orders_with_meta AS
SELECT
    *,
    current_timestamp AS _loaded_at,
    'jaffle_shop' AS _source_system,
    'full' AS _load_type
FROM read_csv_auto(
  'https://raw.githubusercontent.com/dbt-labs/jaffle_shop/main/seeds/raw_orders.csv'
);
""")

conn.execute("SELECT * FROM bronze.orders_with_meta LIMIT 5").fetchdf()

## Full Load vs Incremental Load 비교

| 구분 | Full Load | Incremental Load |
|------|-----------|------------------|
| 구현 난이도 | 낮음 | 높음 |
| 네트워크/비용 | 데이터 크기에 비례 | 변경분에 비례 |
| 삭제 감지 | 자동 | 별도 처리 필요 |
| 적합한 대상 | 코드 테이블, 소규모 마스터 | 대용량 트랜잭션 |

실무에서는 섞어 쓴다. 코드 테이블은 Full Load, 주문/로그는 Incremental Load.

In [ ]:
# 워터마크 상태 확인
conn.execute("SELECT * FROM bronze.watermarks").fetchdf()

In [ ]:
# 정리
conn.close()